# Langchain basics

In [ ]:
pip install -r ../requirements.txt

## Python-dotenv

In [3]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

True

### LLM Models(Wrappers): GPT-3

In [4]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003', temperature=0.7, max_tokens=512)
print(llm)

OpenAI
Params: {'model_name': 'text-davinci-003', 'temperature': 0.7, 'max_tokens': 512, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}}


In [5]:
output = llm('explain quantum mechanics in one sentence')
print(output)



Quantum mechanics is the physical theory describing the behavior and interaction of matter and energy at the atomic and subatomic level.


In [6]:
print(llm.get_num_tokens("explain quantum mechanics in one sentence"))

7


In [8]:
output =  llm.generate(["... is the capital of France.", "What is the formula for the area of circle?"])

In [9]:
print(output.generations)

[[Generation(text='\n\nParis', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nThe formula for the area of a circle is A = πr², where r is the radius of the circle and π is the mathematical constant approximately equal to 3.14.', generation_info={'finish_reason': 'stop', 'logprobs': None})]]


In [10]:
# Only one answer - 2nd question
print(output.generations[1][0].text)



The formula for the area of a circle is A = πr², where r is the radius of the circle and π is the mathematical constant approximately equal to 3.14.


In [11]:
# Only one answer - 1st question
print(output.generations[0][0].text)



Paris


In [42]:
# number of generations
len(output.generations)

2

In [12]:
output =  llm.generate(["Write an original taglines for burger restaurant"] * 3)

In [13]:
print(output)

generations=[[Generation(text='\n\n"Taste the Difference at Our Burger Joint - Satisfaction Guaranteed!"', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\n"A Bite Above the Rest - Burgers with a Delicious Twist!"', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\n"Taste the Juiciness of our Burgers - Satisfaction Guaranteed!"', generation_info={'finish_reason': 'stop', 'logprobs': None})]] llm_output={'token_usage': {'prompt_tokens': 24, 'completion_tokens': 52, 'total_tokens': 76}, 'model_name': 'text-davinci-003'} run=[RunInfo(run_id=UUID('1e75d04e-0c17-4a0e-bf53-c04cadcdaabe')), RunInfo(run_id=UUID('4c179036-880f-4e82-8538-ae13411ee095')), RunInfo(run_id=UUID('5a6c3b6e-9758-4dfa-9be5-bde3daaeaa75'))]


In [14]:
for i in output.generations:
    print(i[0].text)



"Taste the Difference at Our Burger Joint - Satisfaction Guaranteed!"


"A Bite Above the Rest - Burgers with a Delicious Twist!"


"Taste the Juiciness of our Burgers - Satisfaction Guaranteed!"


## ChatModels: GPT-3.5-Turbo and GPT-4

In [15]:
# import schema for the structure called messages
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI

In [16]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5, max_tokens=1024)
messages = [
    SystemMessage(content="You are a physicist and respond only in German."),
    HumanMessage(content="Explain quantum mechanics in one sentence")
]
output = chat(messages)

In [17]:
print(output)

content='Quantenmechanik beschreibt das Verhalten von Teilchen auf sehr kleinen Skalen und ermöglicht Vorhersagen über Wahrscheinlichkeiten von Ereignissen.'


In [18]:
print(output.content)

Quantenmechanik beschreibt das Verhalten von Teilchen auf sehr kleinen Skalen und ermöglicht Vorhersagen über Wahrscheinlichkeiten von Ereignissen.


## Prompt Template
creates dynamic prompts for LLMs

In [19]:
from langchain import PromptTemplate

In [20]:
template = """You are an experienced virologist.
Write a few sentences about the following {virus} in {language}"""

prompt = PromptTemplate(
    input_variables = ["virus", "language"],
    template = template
)
print(prompt)

input_variables=['language', 'virus'] template='You are an experienced virologist.\nWrite a few sentences about the following {virus} in {language}'


In [21]:
from langchain.llms import OpenAI
llm = OpenAI(model_name="text-davinci-003", temperature=0.7)
output = llm(prompt.format(virus='ebola', language='french'))
print(output)

:

L'Ebola est une maladie virale très grave qui se propage par contact direct avec des liquides corporels infectés. Les symptômes comprennent la fièvre, la fatigue, des maux de tête et des douleurs musculaires, des vomissements et de la diarrhée, et peuvent évoluer vers une insuffisance des organes vitaux. La prise en charge et le traitement adéquats sont essentiels pour lutter contre la maladie et sauver des vies.


## Simple Chains
Chains allow us to combine multiple components together to create a simple application

In [22]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain import LLMChain

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)
template = """You are an experienced virologist.
Write a few sentences about the following {virus} in {language}"""


prompt = PromptTemplate(
    input_variables = ["virus", "language"],
    template = template
)

chain = LLMChain(llm=llm, prompt=prompt)
output = chain.run({'virus': "HSV", 'language': 'french'})

In [23]:
print(output)

Le HSV, également connu sous le nom de virus de l'herpès simplex, est un virus à ADN double brin de la famille des Herpesviridae. Il existe deux types d'HSV : le HSV-1, qui provoque généralement des infections buccales et labiales, et le HSV-2, qui est principalement responsable des infections génitales. Ces virus sont hautement contagieux et se transmettent principalement par contact direct avec les lésions cutanées ou les muqueuses infectées. Les symptômes courants incluent des lésions vésiculaires douloureuses, des démangeaisons et des brûlures. Bien qu'il n'existe pas de traitement curatif pour le HSV, des médicaments antiviraux peuvent aider à réduire la fréquence et la gravité des récidives.


## Sequential Chains
Make a series of calls to one or more LLMs. You can take the output from the chain and use it as input to another chain.
There are 2 types of sequential chains:
1. SimpleSequential Chains
2. General form of Sequential chains

Simple sequential chains represent a series of chains, where each individual chain has a single input and a single output, and the output of one step is used as input to the next


In [24]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

llm = OpenAI(model_name='text-davinci-003', temperature=0.7, max_tokens=1024)
prompt1 = PromptTemplate(
    input_variables = ["concept"],
    template = """You are an experienced scientist and python developer.
    Write a function that implements the concept of {concept}"""
)
chain1 = LLMChain(llm=llm, prompt=prompt1)

llm2 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1.2)
prompt2 = PromptTemplate(
    input_variables = ["function"],
    template = """Given the python function {function}, describe it as detailed as possible"""
)
chain2 = LLMChain(llm=llm, prompt=prompt2)

overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose= True)
output = overall_chain.run('linear_regression')




> Entering new SimpleSequentialChain chain...
.

def linear_regression(x, y):
    """
    Implements the concept of linear regression.
    
    Parameters:
    x (list): x-values of the data points
    y (list): y-values of the data points
    
    Returns:
    slope (float): the slope of the best-fit line
    intercept (float): the intercept of the best-fit line
    """
    # Calculate the mean values of x and y
    x_mean = sum(x)/len(x)
    y_mean = sum(y)/len(y)
    
    # Calculate the numerator and denominator of the slope
    numerator = 0
    denominator = 0
    for i in range(len(x)):
        numerator += (x[i] - x_mean) * (y[i] - y_mean)
        denominator += (x[i] - x_mean)**2
    
    # Calculate the slope and intercept
    slope = numerator/denominator
    intercept = y_mean - slope * x_mean
    
    return slope, intercept


This function implements the concept of linear regression, which is a method to determine the relationship between two variables. It takes two lis

## LangChain Agents
Agents are enabling tools for LLMs, then they can do calculations, write code, search the webs or run SQL queries

In [ ]:
pip install langchain_experimental

In [1]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.llms import OpenAI

In [4]:
llm = OpenAI(temperature=0)
# Tools are essentially functions agents can use to interact with outside world.
# It can range from genrtal utilities such as maths calculator, search functions to other chains or other agents
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(), #Tools
    verbose=True)
agent_executor.run('Calculate the square root of the factorial of 20 and display it with 4 decimal points')



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


 I need to calculate the factorial of 20 and then take the square root of that
Action: Python_REPL
Action Input: from math import factorial; print(round(factorial(20)**0.5, 4))
Observation: 1559776268.6285

Thought: I now know the final answer
Final Answer: 1559776268.6285

> Finished chain.


'1559776268.6285'